# LORA Categorizer

In [16]:
import pprint
import re
import os

In [19]:
expCopyPath = "C:\pydev\comfyKrita\ComfyUI\models\loras\\test"
pathTojason = expCopyPath.replace("\\", "/")

print(pathTojason)

C:/pydev/comfyKrita/ComfyUI/models/loras/test


# Categorized file

In [12]:
import re

def create_category_dict(categorized_txt_path):
    category_dict = {}
    with open(categorized_txt_path, 'r') as f:
        current_category = None
        for line in f:
            line = line.strip()
            if line.endswith(':'):  # New category
                current_category = line[:-1] 
            elif line and current_category:
                # Use regex to extract filenames between '<lora:' and ':'
                filenames = re.findall(r'<lora:(.*?):\d+>', line)
                for filename in filenames:
                    category_dict[filename + '.json'] = current_category
    return category_dict

categorized_txt_path = 'C:/pydev/comfyKrita/ComfyUI/models/loras/ponyLora/categorized.txt'  # Replace with your actual path
category_dict = create_category_dict(categorized_txt_path)

# pprint.pprint(category_dict)

{'0580 ada wong fan costume 2_v1_pony.json': 'wearable', 'amputee_XL_pony.json': 'wearable', 'black_woman-000002.json': 'wearable', 'BDSM_PONY.json': 'wearable', 'bigeyes-ponyxl-v1.json': 'wearable', 'bulgeallXLPony.json': 'wearable', 'Concept_Slutty_Clothes.json': 'wearable', 'Concept_X-ray_vision.json': 'wearable', 'Doggystlye_arms_held_back.json': 'wearable', 'Expressive_H-000001.json': 'wearable', 'groping_poses-000009.json': 'wearable', 'handcuffs.json': 'wearable', 'Legs_behind_head-000006.json': 'wearable', 'Maggots-40.json': 'wearable', 'multi_prolapse_ponyXL_v1.json': 'wearable', 'nipple-penetration-ponyxl-v1.json': 'wearable', 'Pain_Fear_and_Desperation_Pony.json': 'wearable', 'ponyxl-choking-concept-v03.json': 'wearable', 'ponyxl_concept-bruised_face.json': 'wearable', 'PonyXL_FAP_CONCEPT_HeadInsertion_ownwaifu.json': 'wearable', 'posture_collar.json': 'wearable', 'PoutyPonyXL_v1.json': 'wearable', 'PulenKompot-Pregnant-PonyXL.json': 'wearable', 'three_holes_knit_ski_mask.js

In [26]:
unique_categories = set(category_dict.values())
print(unique_categories)

{'art style', 'celebrity', 'situations', 'wearable'}


In [17]:
def add_category_to_json_files(json_folder_path, category_dict):
    for filename in os.listdir(json_folder_path):
        if filename.endswith('.json'):
            filepath = os.path.join(json_folder_path, filename)
            with open(filepath, 'r+') as json_file:
                data = json.load(json_file)

                if 'category' not in data:
                    category = category_dict.get(filename, 'uncategorized')  # Get category or 'uncategorized'
                    data['category'] = category
                    json_file.seek(0)
                    json.dump(data, json_file, indent=4)
                    json_file.truncate()

In [24]:
json_folder_path = 'C:/pydev/comfyKrita/ComfyUI/models/loras/test'  # Replace with your actual path 

add_category_to_json_files(json_folder_path, category_dict)

In [38]:
import ipywidgets as widgets
import json
import os
import re

def create_category_dict(categorized_txt_path):
    category_dict = {}
    with open(categorized_txt_path, 'r') as f:
        current_category = None
        for line in f:
            line = line.strip()
            if line.endswith(':'):  # New category
                current_category = line[:-1] 
            elif line and current_category:
                filenames = re.findall(r'<lora:(.*?):\d+>', line)
                for filename in filenames:
                    category_dict[filename + '.json'] = current_category
    return category_dict

def find_uncategorized_json_files(json_folder_path):
    uncategorized_files = []
    for filename in os.listdir(json_folder_path):
        if filename.endswith('.json'):
            filepath = os.path.join(json_folder_path, filename)
            with open(filepath, 'r') as json_file:
                data = json.load(json_file)
                if 'category' not in data or data['category'] == 'uncategorized':
                    uncategorized_files.append(filename)
    return uncategorized_files

def update_json_file(filename, new_category):
    filepath = os.path.join(json_folder_path, filename)
    with open(filepath, 'r+') as json_file:
        data = json.load(json_file)
        data['category'] = new_category
        json_file.seek(0)
        json.dump(data, json_file, indent=4)
        json_file.truncate()

def on_next_button_click(button):
    global current_file_index

    if current_file_index < len(uncategorized_files):
        filename = uncategorized_files[current_file_index]
        filename_label.value = filename
        # category_input.value = '' 
        current_file_index += 1
        category_input.focus()
    else:
        filename_label.value = "No more uncategorized files!"
        category_input.disabled = True
        next_button.disabled = True

json_folder_path = 'path/to/your/json/folder'  # Replace with your actual path
categorized_txt_path = 'path/to/categorized.txt'  # Replace with your actual path
json_folder_path = 'C:/pydev/comfyKrita/ComfyUI/models/loras/test'
categorized_txt_path = 'C:/pydev/comfyKrita/ComfyUI/models/loras/ponyLora/categorized.txt'
category_dict = create_category_dict(categorized_txt_path)

uncategorized_files = find_uncategorized_json_files(json_folder_path)
current_file_index = 0

filename_label = widgets.Label(value=uncategorized_files[current_file_index])
category_input = widgets.Text(placeholder='Enter new category')
next_button = widgets.Button(description='Next')
next_button.on_click(on_next_button_click)

category_input.continuous_update = False 

def on_category_input_change(change):
    if change['new']: 
        new_category = change['new']
        filename = filename_label.value
        update_json_file(filename, new_category)
        on_next_button_click(None) 
        

category_input.observe(on_category_input_change, names='value')

display(filename_label, category_input, next_button)

# {'art style', 'celebrity', 'situations', 'wearable'}

Label(value='defeated after sex xl v1 goofy_2.json')

Text(value='', continuous_update=False, placeholder='Enter new category')

Button(description='Next', style=ButtonStyle())